## 구글 이미지 스크래핑

### Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium_browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium_browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 75.7 MB/s 
     |████████████████████████████████| 356 kB 70.1 MB/s 
     |████████████████████████████████| 3.6 MB 52.7 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

### 라이브러리 import

In [2]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

### `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [7]:
def click_and_save(dir_name, index, img, img_list_length):
  global scraped_count

  try:
    img.click()
    wd.implicitly_wait(3)
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.png')
      print(f'{index+1}/{img_list_length} PNG 이미지 저장')
    else:
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + 'jpg')
      print(f'{index+1}/{img_list_length} JPG 이미지 저장')
    
    scraped_count += 1
  except HTTPError as e:
    print(e)
    pass


### `scroll_down()`: 스크롤을 내리는 함수

In [22]:
def scroll_down():
  scroll_count = 0
  print('[scroll_down(): 스크롤 다운 시작]')

  last_height = wd.execute_script('return document.body.scrollHeight')
  after_click = False
  while True:
    print(f'[스크롤 다운: {scroll_count}]')
    wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    scroll_count += 1
    time.sleep(1)

    new_height = wd.execute_script('return document.body.scrollHeight')

    if last_height == new_height:
      if after_click is True:
        break
      else:
        try:
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input')
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break

    last_height = new_height

### `scraping()`: 구글 이미지 스크래핑 함수

In [40]:
def scraping(dir_name, query):
  global scraped_count
  
  url = f'https://www.google.com/search?q={query}&tbm=isch&ved=2ahUKEwj7jruB_8L1AhVNd94KHZaTAJAQ2-cCegQIABAA&oq=sea&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyCAgAEIAEELEDMgsIABCABBCxAxCDATIICAAQgAQQsQMyCAgAELEDEIMBMgUIABCABDIFCAAQgAQyBQgAEIAEMgsIABCABBCxAxCDATIFCAAQgARQAFgAYLwGaABwAHgAgAGEA4gBhAOSAQMzLTGYAQCqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=wbrqYbuqLc3u-QaWp4KACQ&bih=655&biw=767'
  wd.get(url)
  wd.maximize_window()

  scroll_down()

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  print(div)
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
  print(img_list)
  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script('window.scrollTo(0, window.scrollY + 100')
    except NoSuchElementException as e:
      print(e)
      wd.execute_script('window.scrollTo(0, window.scrollY + 100')
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionResetError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      pass

  try:
    print('[스크래핑 종료 (성공률: %.2ㄹ%%)]' % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)

  wd.quit()

### `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [9]:
def filter_and_remove(dir_name, query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f'{index} 이미지 제거')
        filtered_count += 1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filtered_count += 1

  print(f'[이미지 제거 개수: {filtered_count}/{scraped_count}]')

### 스크래핑 실행

In [ ]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver', options=chrome_options)
scraped_count = 0

path = './'
query = input('검색어 입력: ')

dir_name = path + query
os.makedirs(dir_name)
print(f'[{dir_name} 디렉토리 생성]')

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400)

In [15]:
!ls

drive  sample_data  sea


In [41]:
# 에러 수정 후 다시 실행하기 위해 생성된 sea 디렉토리 삭제
!rmdir sea